In [91]:
# This notebook gathers the data for games that are about to take place, runs them against the models (these must be deployed separately using SageMaker AutoPilot) and then places any bets desired.

In [92]:
# Import the necessary libraries
%pip install ohmysportsfeedspy
from ohmysportsfeedspy import MySportsFeeds #Wrapper provided by my score provider: https://github.com/MySportsFeeds/mysportsfeeds-python
import numpy as np
import pandas as pd
import boto3
from boto3.dynamodb.conditions import Attr
from boto3.dynamodb.conditions import Key
from datetime import datetime
from pytz import timezone
from datetime import timedelta
import time
import pytz

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [93]:
#Setup connection to MySportsFeeds (abbreviated MSF going forward), loop through to get each week's data and then format it.
#TODO: Leverage parameter store to hold the key.

#msf_key = input('MSF Key to Use: ') #Since I'll be storing this publicly I made this an input... you can register for your own key at mysportsfeeds.com.
ssm_client = boto3.client('ssm')
msf_key = ssm_client.get_parameter(Name='MSF_AI_KEY', WithDecryption=True)['Parameter']['Value']

msf = MySportsFeeds('2.1',verbose=False)
msf.authenticate(msf_key, 'MYSPORTSFEEDS')

In [94]:
#Gather the basic game data and score for all of the games.

current_week = 18
tz_import = pytz.timezone('Etc/Zulu')
tz_target = pytz.timezone('US/Eastern')

#Get the date we're going to need for the import.
now = datetime.now()
now_url = '' + now.strftime("%Y") + now.strftime("%m") + now.strftime("%d")  

#Get the JSON data for the week.
msf_games = msf.msf_get_data(league='nfl',season=('2022-playoff'),feed='weekly_games',format='json',week='20')
#msf_games = msf.msf_get_data(league='nfl',season=('2021-2022-regular'),feed='daily_games',format='json',date=now_url)


#Flatten the JSON data and create a data frame.
game_data = pd.json_normalize(msf_games, record_path = ['games'])


In [95]:
#Format the data to have good names and be just the fields that I need.
game_data = game_data.filter(['schedule.id', 'schedule.awayTeam.id', 'schedule.homeTeam.id', 'schedule.venueAllegiance', 'score.awayScoreTotal','score.homeScoreTotal','schedule.week',
    'schedule.originalStartTime','schedule.startTime'])

game_data.rename(columns={"schedule.id": "game_id", 'schedule.awayTeam.id': 'awayTeam_ID', 'schedule.homeTeam.id': 'homeTeam_ID',
    'schedule.venueAllegiance': 'venueAliegiance', 'score.awayScoreTotal': 'awayScore', 'score.homeScoreTotal': 'homeScore',
    'schedule.originalStartTime': 'originalStartTime', 'schedule.startTime': 'startTime', 'schedule.week': 'week'}, inplace=True)     

In [96]:
print(game_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   game_id            4 non-null      int64 
 1   awayTeam_ID        4 non-null      int64 
 2   homeTeam_ID        4 non-null      int64 
 3   venueAliegiance    4 non-null      object
 4   awayScore          0 non-null      object
 5   homeScore          0 non-null      object
 6   week               4 non-null      int64 
 7   originalStartTime  0 non-null      object
 8   startTime          4 non-null      object
dtypes: int64(4), object(5)
memory usage: 416.0+ bytes
None


In [97]:
#Add the week number to each game.  It will be necessary because it is a sort key in my dynamoDB table.
#Also, remove any games that don't start in the next 30 minutes.
tz_import = pytz.timezone('Etc/Zulu')
tz_target = pytz.timezone('US/Eastern')
target_data = pd.DataFrame()
now=now.astimezone(tz_target)

first = True
for index in game_data.index:
    start_time = datetime.strptime(game_data['startTime'].values[index], '%Y-%m-%dT%H:%M:%S.000Z').replace(tzinfo=tz_import)
    start_time = start_time.astimezone(tz_target)
    last_start = start_time - timedelta(minutes = 400)
    
    if(start_time > now) and (last_start < now):
        print('Yes')
        if first:
            print ('Did it')
            target_data = game_data.loc[[index]]
            first = False
        else:
            target_data = target_data.append(game_data.loc[[index]])
        
target_data.reset_index(drop=True, inplace=True)
game_data = target_data.copy()


Yes
Did it
Yes


In [98]:
print(game_data.head())

   game_id  awayTeam_ID  homeTeam_ID venueAliegiance awayScore homeScore  \
0    71075           57           67            HOME      None      None   
1    71076           78           62            HOME      None      None   

   week originalStartTime                 startTime  
0    20              None  2022-01-22T21:30:00.000Z  
1    20              None  2022-01-23T01:15:00.000Z  


In [99]:
#Add the line and the over_under.  Note that this data is coming from the actuals that I have stored in the database used by the actual game.
over_unders = []
home_lines = []
index = 0
for index in game_data.index:
    the_game_id = int(game_data['game_id'].values[index])
    the_week = int(game_data['week'].values[index])
    dynamodb = boto3.resource('dynamodb')
    games_table = dynamodb.Table('LTHOIprod_games')
    lines = games_table.get_item(Key={'Week': the_week, 'Game_Id': the_game_id})
    lines = lines['Item']
    over_under_line = lines['Over_Under']
    home_line = lines['Home_Line']
    over_unders.append(float(over_under_line))
    home_lines.append(float(home_line))

game_data['over_under'] = over_unders
game_data['home_line'] = home_lines

In [100]:
#Add the average points scored and average points against
index = 0
home_team_average_for = []
home_team_average_against = []
away_team_average_for = []
away_team_average_against = []

for index in game_data.index:
    #Home Team Averages
    count = 0
    total_for = 0
    average_for = 0
    total_against = 0
    average_against = 0
    week_to_process = int(game_data['week'].values[index])
    msf_games = msf.msf_get_data(league='nfl',season='2021-regular',feed='seasonal_games',format='json',team=game_data['homeTeam_ID'].values[index])
    msf_games = msf_games['games']
    for game in msf_games:
        if game['schedule']['week'] < week_to_process:
            count = count + 1
            if game['schedule']['homeTeam']['id'] == game_data['homeTeam_ID'].values[index]:
                total_for = total_for + game['score']['homeScoreTotal']
                total_against = total_against + game['score']['awayScoreTotal']
            else:
                total_for = total_for + game['score']['awayScoreTotal']
                total_against = total_against + game['score']['homeScoreTotal']
                
    
    home_team_average_for.append((total_for / count))
    home_team_average_against.append((total_against / count))


    #Away Team Averages
    count = 0
    wins = 0
    losses = 0
    total_for = 0
    average_for = 0
    total_against = 0
    average_against = 0
    msf_games = msf.msf_get_data(league='nfl',season='2021-regular',feed='seasonal_games',format='json',team=game_data['awayTeam_ID'].values[index])
    msf_games = msf_games['games']
    for game in msf_games:
        if game['schedule']['week'] < week_to_process:
            count = count + 1
            if int(game['schedule']['homeTeam']['id']) == int(game_data['awayTeam_ID'].values[index]):
                total_for = total_for + game['score']['homeScoreTotal']
                total_against = total_against + game['score']['awayScoreTotal']
            else:
                total_for = total_for + game['score']['awayScoreTotal']
                total_against = total_against + game['score']['homeScoreTotal']
    
    away_team_average_for.append((total_for / count))
    away_team_average_against.append((total_against / count))
    
    #Pause 5 seconds so we don't exceed the rate limit on the API
    time.sleep(5)


game_data['home_team_average_for'] = home_team_average_for
game_data['home_team_average_against'] = home_team_average_against
game_data['away_team_average_for'] = away_team_average_for
game_data['away_team_average_against'] = away_team_average_against

In [101]:
#Add the number of people betting on each side
the_over_bettors = []
the_under_bettors = []
the_home_bettors = []
the_away_bettors = []

for index in game_data.index:
    the_game_id = int(game_data['game_id'].values[index])
    dynamodb = boto3.resource('dynamodb')
    bets_table = dynamodb.Table('LTHOIprod_bets')
    all_bets = bets_table.query(KeyConditionExpression=Key('LS_Id').eq('d0ab7687-68eb-4753-a595-3a8acc50f3c0'), FilterExpression=Attr('Game_Id').eq(the_game_id) & Attr('Parent_Bet_Id').not_exists())
    home_bettors = 0
    away_bettors = 0
    over_bettors = 0
    under_bettors = 0
    for bet in all_bets['Items']:
        if ('Line_Bet' in bet and bet['Line_Bet'] == 1):
            home_bettors = home_bettors + 1
        elif ('Line_Bet' in bet and bet['Line_Bet'] == -1):
            away_bettors = away_bettors + 1
        elif (bet['Over_Under_Bet'] == 1):
            over_bettors = over_bettors + 1
        elif (bet['Over_Under_Bet'] == -1):
            under_bettors = under_bettors +1

    the_over_bettors.append(over_bettors)
    the_under_bettors.append(under_bettors)
    the_home_bettors.append(home_bettors)
    the_away_bettors.append(away_bettors)
    
game_data['over_bets'] = the_over_bettors
game_data['under_bets'] = the_under_bettors
game_data['home_bets'] = the_home_bettors
game_data['away_bets'] = the_away_bettors

In [102]:
import pytz

#Find what the actual final line was.
the_final_line = []
the_final_ou = []
start_time = datetime.now()
tz_import = pytz.timezone('Etc/Zulu')
tz_target = pytz.timezone('US/Eastern')


#for index in game_data.index:
#    the_game_id = int(game_data['game_id'].values[index])
#    start_time.replace(tzinfo=tz_import)

    #Convert Start time to Date Object and Set Date String for MSF Query
#    start_time = datetime.strptime(game_data['startTime'].values[index], '%Y-%m-%dT%H:%M:%S.000Z').replace(tzinfo=tz_import)
#    start_time = start_time.astimezone(tz_target)
#    url_start_time = '' + start_time.strftime("%Y") + start_time.strftime("%m") + start_time.strftime("%d")                                                                                             

    #If there is an originalStartTime then over-ride the MSF Query String
#   if game_data['originalStartTime'].values[index] is not None:
#        original_start_time = datetime.strptime(game_data['originalStartTime'].values[index], '%Y-%m-%dT%H:%M:%S.000Z').replace(tzinfo=tz_import)
#        original_start_time = original_start_time.astimezone(tz_target)
#        url_start_time = '' + original_start_time.strftime("%Y") + start_time.strftime("%m") + start_time.strftime("%d")
#        print('{} used {} instead.'.format(the_game_id, url_start_time))

#    print(url_start_time)
#    msf_lines = msf.msf_get_data(league='nfl',season='2021-2022-regular',feed='daily_game_lines',format='json',game=the_game_id,date=url_start_time)
#    msf_spreads = msf_lines['gameLines'][0]['lines'][0]['pointSpreads']
#    msf_ous = msf_lines['gameLines'][0]['lines'][0]['overUnders']

#    closest_to_kickoff = datetime(1970, 1, 1).replace(tzinfo=tz_import)
#    line = 0
#    for spread in msf_spreads:
#        current_time = datetime.strptime(spread['asOfTime'], '%Y-%m-%dT%H:%M:%S.000Z').replace(tzinfo=tz_import)
#        current_time = current_time.astimezone(tz_target)
#        if (current_time > closest_to_kickoff) and (current_time < start_time) and (spread['pointSpread']['gameSegment'] == 'FULL'):
#            closest_to_kickoff = current_time
#            line = spread['pointSpread']['homeSpread']

#    closest_to_kickoff = datetime(1970, 1, 1).replace(tzinfo=tz_import)
#    ou_final = 0
#    for ou in msf_ous:
#        current_time = datetime.strptime(ou['asOfTime'], '%Y-%m-%dT%H:%M:%S.000Z').replace(tzinfo=tz_import)
#        current_time = current_time.astimezone(tz_target)
#        if (current_time > closest_to_kickoff) and (current_time < start_time) and (ou['overUnder']['gameSegment'] == 'FULL'):
#            closest_to_kickoff = current_time
#            ou_final = ou['overUnder']['overUnder']

#    the_final_line.append(line)
#    the_final_ou.append(ou_final)

#    time.sleep(20)

#game_data['final_line'] = the_final_line
#game_data['final_over_under'] = the_final_ou

game_data['final_line'] = game_data['home_line']
game_data['final_over_under'] = game_data['over_under']



In [103]:
#Trim the start times because they don't matter.  They were only in the data to assist with querying the database.
all_data = game_data.copy()
game_data.drop(labels=['originalStartTime', 'startTime'], axis=1, inplace=True)

In [104]:
print(game_data.head())
print(game_data.tail())

   game_id  awayTeam_ID  homeTeam_ID venueAliegiance awayScore homeScore  \
0    71075           57           67            HOME      None      None   
1    71076           78           62            HOME      None      None   

   week  over_under  home_line  home_team_average_for  \
0    20        47.5       -3.5              24.647059   
1    20        47.0       -6.0              26.470588   

   home_team_average_against  away_team_average_for  \
0                  20.823529              27.058824   
1                  21.823529              25.117647   

   away_team_average_against  over_bets  under_bets  home_bets  away_bets  \
0                  22.117647          0           0          1          1   
1                  21.470588          0           0          1          1   

   final_line  final_over_under  
0        -3.5              47.5  
1        -6.0              47.0  
   game_id  awayTeam_ID  homeTeam_ID venueAliegiance awayScore homeScore  \
0    71075           57

In [105]:
#Trim the week
game_data.drop(labels=['week'], axis=1, inplace=True)

In [106]:
game_data.drop(labels=['homeScore', 'awayScore', 'game_id', 'venueAliegiance'], axis=1, inplace=True)

In [107]:
print(game_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   awayTeam_ID                2 non-null      int64  
 1   homeTeam_ID                2 non-null      int64  
 2   over_under                 2 non-null      float64
 3   home_line                  2 non-null      float64
 4   home_team_average_for      2 non-null      float64
 5   home_team_average_against  2 non-null      float64
 6   away_team_average_for      2 non-null      float64
 7   away_team_average_against  2 non-null      float64
 8   over_bets                  2 non-null      int64  
 9   under_bets                 2 non-null      int64  
 10  home_bets                  2 non-null      int64  
 11  away_bets                  2 non-null      int64  
 12  final_line                 2 non-null      float64
 13  final_over_under           2 non-null      float64
dty

In [108]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
import sagemaker
import uuid
session = sagemaker.Session()

away_endpoint = 'away-bets8'
home_endpoint = 'home-bets8'
over_endpoint = 'over-bets8'
under_endpoint = 'under-bets8'


away_predictor = Predictor(endpoint_name=away_endpoint, sagemaker_session=session, serializer=CSVSerializer(), deserializer=CSVDeserializer())
home_predictor = Predictor(endpoint_name=home_endpoint, sagemaker_session=session, serializer=CSVSerializer(), deserializer=CSVDeserializer())
over_predictor = Predictor(endpoint_name=over_endpoint, sagemaker_session=session, serializer=CSVSerializer(), deserializer=CSVDeserializer())
under_predictor = Predictor(endpoint_name=under_endpoint, sagemaker_session=session, serializer=CSVSerializer(), deserializer=CSVDeserializer())

def place_bet(game_id, ou, line):
    table = dynamodb.Table('LTHOIprod_bets')
    print('ou: {}    line: {}'.format(ou, line))
    if (line != 0):
        response = table.put_item(
             Item={
                     'LS_Id': 'd0ab7687-68eb-4753-a595-3a8acc50f3c0',
                     'Bet_Id': str(uuid.uuid4()),
                     'Line_Bet': line,
                     'Bet_Amount': 25,
                     'User_Id': 'd219a2f0-9496-45ad-8c00-d8bf77480a1a',
                     'Game_Id': int(game_id)
             })
        print('line response: {}'.format(response))
    elif (ou != 0):
        response = table.put_item(
             Item={
                     'LS_Id': 'd0ab7687-68eb-4753-a595-3a8acc50f3c0',
                     'Bet_Id': str(uuid.uuid4()),
                     'Over_Under_Bet': ou,
                     'Bet_Amount': 25,
                     'User_Id': 'd219a2f0-9496-45ad-8c00-d8bf77480a1a',
                     'Game_Id': int(game_id)
             })
        print(response)


for index in game_data.index:
    
    just_row = game_data.loc[[index]]
    just_row['bet_is_smart'] = 0
    
    away_bet = away_predictor.predict(just_row.to_csv(sep=",", header=False, index=False))[0][0]
    home_bet = home_predictor.predict(just_row.to_csv(sep=",", header=False, index=False))[0][0]
    over_bet = over_predictor.predict(just_row.to_csv(sep=",", header=False, index=False))[0][0]
    under_bet = under_predictor.predict(just_row.to_csv(sep=",", header=False, index=False))[0][0]
    
    print(all_data['game_id'][index])
    
    home_bet = int(home_bet)
    away_bet = int(away_bet)
    over_bet = int(over_bet)
    under_bet = int(under_bet)
    
    
    if (home_bet==1) and (away_bet==0):
        print('home')
        place_bet(all_data['game_id'][index], 0, 1)
    elif (away_bet==1) and (home_bet==0):
        print('away')
        place_bet(all_data['game_id'][index], 0, -1)
        
    if (over_bet==1) and (under_bet==0):
        print('over')
        place_bet(all_data['game_id'][index], 1, 0)
    elif (over_bet==0) and (under_bet==1):
        print('under')
        place_bet(all_data['game_id'][index], -1, 0)
        

71075
home
ou: 0    line: 1
line response: {'ResponseMetadata': {'RequestId': 'QGC98EG8RKG2EO41IF86V170RJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 22 Jan 2022 21:15:46 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'QGC98EG8RKG2EO41IF86V170RJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}
over
ou: 1    line: 0
{'ResponseMetadata': {'RequestId': 'ET0MEAQ18HD262OL1FKV9LFN1JVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 22 Jan 2022 21:15:46 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'ET0MEAQ18HD262OL1FKV9LFN1JVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}
71076
home
ou: 0    line: 1
line response: {'ResponseMetadata': {'RequestId': 'H2U3JA3UT1BON87RHFQIFL33VJ